In [1]:
import blocksci
import collections
import pandas as pd
import numpy as np
from pathlib import Path


parser_data_directory = Path("/mnt/anal/config.json")
cluster_directory = Path("/mnt/anal/cluster/")
dumplings_directory = Path("/mnt/dumplings/")

chain = blocksci.Blockchain(str(parser_data_directory))

from typing import Tuple

def get_block_height_for_date(date: str) -> int:
    return chain.range(date)[0].height

def get_block_height_range(start: str, end: str) -> Tuple[int, int]:
    return get_block_height_for_date(start), get_block_height_for_date(end)

# Whirlpool clusters
`9ea3e5d217c76e6be6ace7fe4fb3622976103466de81d2b02cd87c54f9e91a2f` should have 21 clustered addresses by the simple one output heuristics
`00ab23721356a159b695060b3975469fa0703e55398e9123ef59d2a5b94b1058` should have 567 addresses clustered

In [9]:
# import random

one_random_wp_cj = chain.tx_with_hash("9ea3e5d217c76e6be6ace7fe4fb3622976103466de81d2b02cd87c54f9e91a2f")
# one_random_wp_cj = chain.tx_with_hash("00ab23721356a159b695060b3975469fa0703e55398e9123ef59d2a5b94b1058")
address_im_looking_for = one_random_wp_cj.outputs[1].spending_tx.outputs[0].address
print(address_im_looking_for)

start = one_random_wp_cj.block_height
stop = one_random_wp_cj.block_height + 1

start = 0
stop = len(chain)

# clusterer = blocksci.cluster.ClusterManager(str(cluster_directory / "fr"), chain)

%time clusterer = blocksci.cluster.CoinjoinClusterManager.create_clustering(  \
    chain=chain, \
    start=start, \
    stop=stop, \
    output_path="/mnt/anal/clustering/whirlpool_out/", \
    overwrite=True, \
    coinjoin_type="wasabi2", \
    heuristics=["OneOutputConsolidation", "OneInputConsolidation"] \
)


bc1qxvsjlwnrhel0elpcv0tmhflrxl7t7hphgau3pf
Using heuristic: None
Using heuristic: OneOutputConsolidation
Using heuristic: OneInputConsolidation
Collected 16076035 addresses
CPU times: user 3min 43s, sys: 14.5 s, total: 3min 58s
Wall time: 1min 2s


In [22]:
# import random

one_random_wp_cj = chain.tx_with_hash("9ea3e5d217c76e6be6ace7fe4fb3622976103466de81d2b02cd87c54f9e91a2f")
one_random_wp_cj = chain.tx_with_hash("00ab23721356a159b695060b3975469fa0703e55398e9123ef59d2a5b94b1058")
one_random_wp_cj = chain.tx_with_hash("021dc91d5d9d74babeed5dbf12a6287cbc4a46317afa357b45c46776940cf902")
address_im_looking_for = one_random_wp_cj.outputs[1].spending_tx.outputs[0].address
print(address_im_looking_for)

start = one_random_wp_cj.block_height
stop = one_random_wp_cj.block_height + 91302

# start = 0
# stop = len(chain)

# clusterer = blocksci.cluster.ClusterManager(str(cluster_directory / "fr"), chain)

%time clusterer = blocksci.cluster.CoinjoinClusterManager.create_clustering(  \
    chain=chain, \
    start=start, \
    stop=stop, \
    output_path="/mnt/anal/clustering/whirlpool_out_threshold/", \
    overwrite=True, \
    coinjoin_type="whirlpool", \
    heuristics=["OneOutputConsolidation", "OneInputConsolidation", "TwoHopOutputThresholdConsolidation", "ThreeHopOutputThresholdConsolidation"] \
)


bc1qttdxayfzyup2gaw4dkdjz0smlwylmtzpy6a3e9
Using heuristic: None
Using heuristic: OneOutputConsolidation
Using heuristic: OneInputConsolidation
Using heuristic: TwoHopOutputThresholdConsolidation
Using heuristic: ThreeHopOutputThresholdConsolidation
Collected 2991384 addresses
Creating disjoint sets of size 2991384
Created disjoint sets of size 2991384
Preparing to serialize cluster data
Serializing cluster data
Serialized
CPU times: user 1min 47s, sys: 7.44 s, total: 1min 54s
Wall time: 20 s


In [ ]:
clusterer = blocksci.cluster.ClusterManager("/mnt/anal/clustering/wasabi1/", chain)

print(len(clusterer.clusters()))
%time bigger_than_one = [cluster for cluster in clusterer.clusters() if cluster.address_count() > 1]
print(len(bigger_than_one))
%time bigger_than_one.sort(key=lambda c: c.balance(), reverse=True)
cluster = bigger_than_one[0]


In [6]:
a = bigger_than_one[1].addresses.to_list()

s = {"3H7rzruh2wLkCF2pVfn3v1po6aWES2f7iB", "39SZ4LKMuKy2vJ63r46yqnTub7ciAvY2B3", "3NrUFDF1uqLi7d5UHUD7qoxZcAwLszbXpT", "3FyGsPeKRXSazXMre5zvcwFqVrc86gNbLP", "3EtZBuGSa8oUT7jxo4S9Bqws24eJ8cgP24", "38dsUSUSVw7S2cwX81R9Ex2yqUPGU7EGSE", "3GHuXuCJpesrCMvVSeVQatXgxod6f9jq65", "3KyWzoPbWyvBVekauAd96h5LsrJbDTVs48", "3NqG5tio4G2YeLPtzWSVD9YWNL7Mx3Af5W", "32mFyKRHGmSSG8ELvnr8eUW5y2CEroFWRt", "3AFjEf7e2uiq2kyZJouASEn3hdxGQwQhAs", "3HJasiWwPipfheoPWw4b2KCWGYJnfyESsS", "3H9ECmG59doCbsGwT2U9ipH2zKDXc33Mb1", "3NCyJpLcbCntj4ycAa6mPwzDhDx7raHkW4", "3EfmDALTK4HL6UAfZbSTkCY6KmTpBDkMd3", "34GDGGvvmas8vZvspL4xoJDbnjcVVSyDGg", "3E865Edvwvv2f9SadroyPkuD8NWsnnh12h", "3Gxxz3zwRPSiwJELLn12VuXCdWUSuc4Jqe", "333rRogJJhnbhTZWxH7ahx7mgpBnvSh77k", "3BZDdyUPbg1eNTV6R9xJYUsEq6NR6rj2rE", "32thrw17kHJcAeUgeMEuuwxn2WZzF1UZqe", "35UzUksackYfJYeGg3RBSkKpx53YJQshCQ", "379gaVDJ8QKJ3JZpqsd54w2YAU9dsezyS4", "36AxQ412DjtZYiaGHWBS2yhtLYzJTiG5ip", "3BDaQUuSEfwmGTLeJZmUYtaeWWuAmrYoBv", "3B8H61BWrbt2XgNYKjqeUbteXxz5ZfVzwK", "35Bvp5N9qjp4Cxr7ntmEQv5rngZhe1RRDx", "36Q8uRcVpcvkGjfDbNBESmKtYygtSVkq6h", "3D4GWmZXSzvqkZR18vfJoW6HKqHCp6RM6a", "3Jy9bQ3xaXWSrVfmyrNSrhQjxKwogSTphF", "37y7iY4aLD5kQ4hf4zGYgD5dHqJK9z5K6v", "3QSbqjR4aWAeNxgZyvY4G59KPpLiZkSobL", "361nuo9aqPvsEMNZ23ZEgZooWdoNogmBWA", "39nWFQ9UjFxSuCU2rQo2eTr7HpA8TvHGuA", "3KCUnnrKeBnZDaHFi4HzimAPjc6BiXFoPr", "3JrqkJz6rC9Hcd5qjm8XJbqYFga9nN4GAs", "38QQbdgboThzMsuK1YzSL5nEVjZvdLBhWC", "3QDQeyFuYvumJjUThNFwUWEQHmwnaja2T4", "3Bwj1esSNx9qwkJyYPRqi4eZ9puzNWDgN3", "3CmTJW1rsCGdiDrabCL2SJo6i84Htwk7M3", "3MQu2hzCfQM33Xz6PN6rg9jRTZvrYVeU2R", "38ihLYWdpjDyrBeuu5ndps8SWipmPHcDE6", "3QYzQ1LCRdNFbupM9QipMQnZCfe6dh8PEY", "3AMPWEG7rpRzc2mtNAxN9E1CD4Vff7ux1d", "3G72zVkYH1PD2HbMKvVHjmrVZiMMpaJGeS", "3EQjsa37d1ePVCBR8TfiAZ9WhbYGypeWEA", "3GP87BRQtvzTkVPKzhXgVnft9QpZrZRUxr", "3L58gWsGyizovHfzUgXiaKsJRR5mdFJ2XD", "3LxEMPDjhcsyrC9Dk1mVpn4AFmLG5X97VV", "3PVbHejz8ttTUqHTBAYjzdgLBcsDLkH3fN", "3AJAMUtRyZE6SNFRfY19dFcFFs8T6rG8ZJ", "3GG32oH3kaU8WYK38vuwesUbSoq9KSzVAf", "353bvSADaPRnr8owqQzkysN1wPCPLvj6fn", "343sJ9MfCnn3v1zo4minZzhM8UcqCxEW8y", "37zdULp8aHmkHFuP3vts3DwGEEZtvuM7t6", "3Jv8kK6g8izMUZYQdRBX18mVKTjrQGNmQm", "33W8F8ufZEwmRS1H9eSKdfo4KyvPw4Rt7c", "3EMSoNRT6JVNVU5eeMJBtQgQxPQeGrAkJs", "3BPp7i6GJt8KaRMqLFxEg4HrKQWGg74vvP", "3FJ8nouCg3JHBPw9B9f6WDCWsQ8UrHn5rb", "3Q1tVt81o7zp3BVP6UY2JAXZonSB9PFBWy", "39iPMkYdxscXmePnjur3VpKU2JPG8ZKTJ1", "38CfEdeAYvBNzkCrUPAXT8BT4k8pXQUpMR", "3B6otXTi8ekZgY4VFybmPv2j9LToHFNHDt", "3P2fwiPnQxtZR2ba8J4EDsNLAqu5SEKL4W", "3ESJdTRKCqDbKu7tG2jCeCbSw59gTzA1GZ", "3FsdGgGQUVWbLwqRQpHfa3xL68XqxGnXcX", "3PoN4wXzLy61b3u2585b2UwkwSxFnVNYDv", "3KqrrwY83Fy2hCnVB1EFF74xvXVmaHuW79", "33FNudmSNr3wYb6jyaPS2yURar2SunNxNy", "3J4JoKWapWwK7PoSee3gQzgDiVVhViYAYJ", "3AhsUnWu7hwNHyntHZZmTU65wihh9bEfd6", "32y1ogyNNvjHo87D5pT48j8G8ppGDeYY69", "38UXvHchd1HRohgspQaHQeCLt2M5gXatKg", "3AFjEf7e2uiq2kyZJouASEn3hdxGQwQhAs", "3FR3diXoNLUhbW3mFC9ZDSVwkyjtPCU8Fn", "343Q125WjCnLjC9JKQUsVzd3t5u3fbVniA", "3FjcAVpQPfPiyGDLjTr1v4xhGaFR1nX3mY", "3P8z2jMTzuwpV5JAzL8FWZ61VXXN2se4me", "3F8L9EVFb5oF8HkLwRbPJeALbMhCZx6rcy"}
print(a)
print(len(s), len(a))

[PubkeyHashAddress(1F3Cpgben5uRAMptnPRL9coAbKp9YmWqfb), WitnessPubkeyAddress(bc1qn878vf8699p32y3ehq2wwaq876uu7yye5m5ahc)]
79 2


# Wasabi1 clustering


In [23]:
clusterer = blocksci.cluster.CoinjoinClusterManager("/mnt/anal/clustering/whirlpool_out_threshold/", chain)
one_random_wp_cj = chain.tx_with_hash("021dc91d5d9d74babeed5dbf12a6287cbc4a46317afa357b45c46776940cf902")
address_im_looking_for = one_random_wp_cj.outputs[3].address

# consolidation: blocksci.cluster.Cluster = clusterer.cluster_with_address(chain.address_from_string("bc1qfpcuqtv6739p3v9f6v3ahgxn2z00lgyaljx899"))
consolidation: blocksci.cluster.Cluster = clusterer.cluster_with_address(address_im_looking_for)


print(consolidation.address_count())
addr = set()
for address in consolidation.addresses.to_list():
    # print(address)
    addr.add(str(address))

print(addr.pop(), "bc1qfpcuqtv6739p3v9f6v3ahgxn2z00lgyaljx899" in addr)

1
bc1q5nat2fx3l3nsu5rql8fzzjqqkue0pz8cu7h72d False
